This script generates code smells using an adapted version of the Pysmell software tool over the buggy data


In [1]:
import os
import subprocess
from subprocess import Popen
import pandas as pd
import numpy as np
import glob
import sys
from pathlib import Path
#getting full path to import adapted version of the Pysmell software tool
cwd = os.getcwd()
path = Path(cwd).parent.parent
toolpath = str(path) + r'\tools\tmpPysmell'
astpath = str(path) + r'\tools\tmpPysmell\ast'
sys.path.append(toolpath)
sys.path.append(astpath)
from pysmell import *




Creates the dataframe where all code smells will be stored, dataframe will be converted to a CSV file later on

In [2]:
bug_smells = pd.DataFrame(columns =['bug_id','bug_type','project_full_name','file_name','function_name','code_smell_metric','code_smell_metric_value'])

In [3]:
bug_smells.head()


,bug_id,bug_type,project_full_name,file_name,function_name,code_smell_metric,code_smell_metric_value


Code_smells_to_csv function - Returns a dataframe with a row having bug_id,bug_type,file_name and smells to be added to bug_smells CSV

In [4]:
bugs = pd.read_csv(r'..\..\subjects\data\generated\bugs-in-quantum-computing-platforms.csv', sep=',')

In [5]:
bugs.shape

(223, 9)

In [6]:
bugs.head()

,project_full_name,project_repository_url,fix_commit_hash,bug_id,bug_type,high_level_buggy_component,bug_pattern,symptom,complexity
0,Qiskit/qiskit-ignis,https://github.com/Qiskit/qiskit-ignis.git,ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666,1,Classical,Quantum State Evaluation,Missing Error Handling,"Crash - OS/PL Error, Crash",19
1,Qiskit/qiskit-ignis,https://github.com/Qiskit/qiskit-ignis.git,e696fee3e52550f50fb240a4c6356c61c9f01725,10,Quantum,Quantum State Evaluation,Incorrect Numerical Computation,Incorrect Output,2
2,Qiskit/qiskit-aer,https://github.com/Qiskit/qiskit-aer.git,dc7a2413e6c373455c00cff51e37eaaa17a4a7ac,1010,Quantum,Optimizations,Barrier Related,"Crash - Application Error, Crash",6
3,Qiskit/qiskit-aer,https://github.com/Qiskit/qiskit-aer.git,7de864cc989b354d0ad1febbd62b891270927b6b,1012,Classical,Testing,"Flaky Test, Misconfiguration, Misconfiguration...",Failing Test,1
4,Qiskit/qiskit-aer,https://github.com/Qiskit/qiskit-aer.git,f3b4303f478ff84bf8da15761827287d3a2f16e4,1019,Quantum,Simulator,"API Misuse - Internal, Typo",Incorrect Output,1


In [7]:
# Converting series to String

bugs['fix_commit_hash']= pd.Series(bugs['fix_commit_hash'], dtype="string")
bugs['bug_id']= pd.Series(bugs['bug_id'], dtype="string")
bugs['bug_type']= pd.Series(bugs['bug_type'], dtype="string")
bugs['project_full_name']= pd.Series(bugs['project_full_name'], dtype="string")

In [8]:
length = bugs.shape[0]
j = 0

for i in range(length):
    sha1 = (bugs['fix_commit_hash'].iloc[i])
    repo_url = 'https://github.com/' + bugs['project_full_name'].iloc[i] +'.git'
    sub_repo_url = bugs['project_full_name'].iloc[i].split('/')
    print(sha1)
    print(repo_url)
    print(sub_repo_url[1])
    
    ### Clone Repository
    process = subprocess.Popen(['git','clone',repo_url],cwd = r'..\..\subjects\tmp-repositories', stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output, errors = process.communicate()

    ### Checkout to the fixed version
    process2 = subprocess.Popen(['git','checkout',sha1+'~1'],cwd = r'..\..\subjects\tmp-repositories' + '\\' + str(sub_repo_url[1]), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output2, errors = process2.communicate()
    
    ### Obtain only the modified files
    
    process3 = subprocess.Popen(['git','diff-tree','--no-commit-id','--name-only','-r',sha1 + '~1'],cwd = r'..\..\subjects\tmp-repositories' + '\\' + str(sub_repo_url[1]), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
    output3, errors = process3.communicate()
    #print(output3.splitlines())
    
    ### todos os ficheiros python de um dado repo
    pyPath = r'..\..\subjects\tmp-repositories' + '\\' + sub_repo_url[1]
    #ficheiros = glob.glob(pyPath + '/**/*.py',recursive=True)
    
    print('Hash = ' + sha1)
    
    for ficheiro in output3.splitlines():
        
        #print(ficheiro)
        idd = str(bugs['bug_id'].iloc[i])
        tipo = bugs['bug_type'].iloc[i]
        ficheiro_final = pyPath + '\\' + ficheiro
        print(ficheiro_final)
                
        try:
            bug_smells = bug_smells.append(Code_smells_to_csv(idd,tipo,ficheiro_final,sub_repo_url[1]),ignore_index = True)
          
           
        except:
            print("An exception occurred. The file is not a python file")
        

ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666
https://github.com/Qiskit/qiskit-ignis.git
qiskit-ignis
Hash = ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666
..\..\subjects\tmp-repositories\qiskit-ignis\.gitignore
..\..\subjects\tmp-repositories\qiskit-ignis\.gitignore
An exception occurred. The file is not a python file
..\..\subjects\tmp-repositories\qiskit-ignis\examples/process-tomography.ipynb
..\..\subjects\tmp-repositories\qiskit-ignis\examples/state-tomography.ipynb
..\..\subjects\tmp-repositories\qiskit-ignis\qiskit_ignis/tomography/basis/circuits.py
state_tomography_circuits
(1, None, 46, 2)
(2, None, 46, 1)
process_tomography_circuits
(1, None, 85, 2)
(2, None, 85, 1)
_tomography_circuits
(1, None, 139, 2)
(2, None, 139, 86)
(11, None, 263, 23, 1, 5)
(6, None, 263, 1)
(11, None, 276, 23, 1, 5)
(6, None, 276, 1)
(11, None, 289, 23, 1, 5)
(6, None, 289, 1)
(6, None, 357, 1)
(6, None, 357, 1)
(6, None, 367, 1)
(6, None, 367, 1)
(6, None, 367, 1)
default_basis
(1, None, 383, 1)
(2, None, 383

Hash = 01196202394aed89f43d3044d64eb9e41f7d583c
..\..\subjects\tmp-repositories\pennylane\.github/CHANGELOG.md
..\..\subjects\tmp-repositories\pennylane\.github/CHANGELOG.md
An exception occurred. The file is not a python file
..\..\subjects\tmp-repositories\pennylane\.github/workflows/tests.yml
..\..\subjects\tmp-repositories\pennylane\.github/workflows/tests.yml
An exception occurred. The file is not a python file
..\..\subjects\tmp-repositories\pennylane\pennylane/devices/default_mixed.py
__init__
(1, None, 89, 3)
(2, None, 89, 6)
_create_basis_state
(1, None, 101, 2)
(2, None, 101, 4)
(6, None, 112, 1)
state
(1, None, 116, 1)
(2, None, 116, 3)
reset
(1, None, 123, 1)
(2, None, 123, 3)
analytic_probability
(1, None, 130, 1)
(2, None, 130, 5)
_get_kraus
(1, None, 143, 2)
(2, None, 143, 5)
_apply_channel
(1, None, 162, 3)
(2, None, 162, 22)
(11, None, 176, 43, 1, 9)
(6, None, 184, 1)
(6, None, 188, 1)
(11, None, 191, 40, 1, 6)
(6, None, 192, 1)
(6, None, 195, 1)
(6, None, 196, 1)
(6, 

..\..\subjects\tmp-repositories\qiskit-aer\src/framework/results/data/subtypes/data_map.hpp
An exception occurred. The file is not a python file
..\..\subjects\tmp-repositories\qiskit-aer\src/framework/results/data/subtypes/list_data.hpp
..\..\subjects\tmp-repositories\qiskit-aer\src/framework/results/data/subtypes/list_data.hpp
An exception occurred. The file is not a python file
..\..\subjects\tmp-repositories\qiskit-aer\src/framework/results/data/subtypes/pybind_data_map.hpp
..\..\subjects\tmp-repositories\qiskit-aer\src/framework/results/data/subtypes/pybind_data_map.hpp
An exception occurred. The file is not a python file
..\..\subjects\tmp-repositories\qiskit-aer\src/framework/results/data/subtypes/pybind_subtypes.hpp
..\..\subjects\tmp-repositories\qiskit-aer\src/framework/results/data/subtypes/pybind_subtypes.hpp
An exception occurred. The file is not a python file
..\..\subjects\tmp-repositories\qiskit-aer\src/framework/results/data/subtypes/single_data.hpp
..\..\subjects\tmp-

Hash = 474f337b176f706a180ac62d6c7840c01f604192
..\..\subjects\tmp-repositories\pennylane\pennylane/tape/tapes/jacobian_tape.py
__init__
(1, None, 87, 1)
(2, None, 87, 1)
_grad_method
(1, None, 91, 2)
(2, None, 91, 11)
(6, None, 125, 2)
(13, None, 138, 2)
(10, None, 141, 31, 1)
(11, None, 143, 20, 1, 5)
_update_gradient_info
(1, None, 148, 1)
(2, None, 148, 4)
(13, None, 152, 2)
(6, None, 155, 1)
(6, None, 157, 1)
_grad_method_validation
(1, None, 159, 2)
(2, None, 159, 12)
(6, None, 182, 1)
(11, None, 185, 92, 2, 15)
(6, None, 186, 1)
(13, None, 187, 2)
(11, None, 192, 48, 2, 10)
(11, None, 197, 47, 2, 10)
numeric_pd
(1, None, 208, 3)
(2, None, 208, 25)
(6, None, 235, 1)
(1, None, 253, 1)
(2, None, 253, 5)
(6, None, 264, 1)
(6, None, 268, 1)
(1, None, 285, 1)
(2, None, 285, 3)
(6, None, 296, 1)
(6, None, 297, 1)
device_pd
(1, None, 304, 3)
(2, None, 304, 7)
analytic_pd
(1, None, 331, 4)
(2, None, 331, 1)
jacobian
(1, None, 347, 3)
(2, None, 347, 48)
(11, None, 449, 47, 1, 7)
(11, None

..\..\subjects\tmp-repositories\qiskit-aer\qiskit/providers/aer/library/save_instructions/save_state.py
save_state
(1, None, 58, 1)
(2, None, 58, 3)
__init__
(1, None, 26, 2)
(2, None, 26, 3)
..\..\subjects\tmp-repositories\qiskit-aer\qiskit/providers/aer/library/save_instructions/save_statevector.py
save_statevector
(1, None, 77, 1)
(2, None, 77, 3)
save_statevector_dict
(1, None, 103, 1)
(2, None, 103, 3)
__init__
(1, None, 23, 2)
(2, None, 23, 1)
__init__
(1, None, 51, 2)
(2, None, 51, 1)
..\..\subjects\tmp-repositories\qiskit-aer\qiskit/providers/aer/library/save_instructions/save_unitary.py
save_unitary
(1, None, 43, 1)
(2, None, 43, 3)
__init__
(1, None, 23, 2)
(2, None, 23, 1)
..\..\subjects\tmp-repositories\qiskit-aer\qiskit/providers/aer/library/save_matrix_product_state.py
..\..\subjects\tmp-repositories\qiskit-aer\qiskit/providers/aer/library/save_matrix_product_state.py
An exception occurred. The file is not a python file
..\..\subjects\tmp-repositories\qiskit-aer\qiskit/pr

...

(6, None, 539, 1)
(6, None, 539, 1)
(6, None, 543, 1)
(6, None, 543, 1)
(6, None, 544, 1)
(6, None, 544, 1)
(6, None, 550, 1)
(6, None, 550, 1)
(6, None, 550, 1)
(6, None, 550, 1)
(6, None, 551, 1)
(6, None, 551, 1)
(6, None, 551, 1)
(6, None, 551, 1)
(6, None, 554, 4, 1)
test_compose_front_both_kraus
(1, None, 594, 1)
(2, None, 594, 18)
(6, None, 596, 2, 1)
(6, None, 597, 2, 1)
(6, None, 598, 2, 1)
(6, None, 599, 2, 1)
(6, None, 610, 2)
(6, None, 611, 2)
(6, None, 612, 2)
test_compose_front_both_unitary
(1, None, 616, 1)
(2, None, 616, 30)
(6, None, 623, 1)
(6, None, 623, 1)
(6, None, 624, 1)
(6, None, 624, 1)
(6, None, 628, 1)
(6, None, 628, 1)
(6, None, 629, 1)
(6, None, 629, 1)
(6, None, 635, 1)
(6, None, 635, 1)
(6, None, 636, 1)
(6, None, 636, 1)
(6, None, 637, 1)
(6, None, 637, 1)
(6, None, 638, 1)
(6, None, 638, 1)
(6, None, 642, 1)
(6, None, 642, 1)
(6, None, 642, 1)
(6, None, 642, 1)
(6, None, 643, 1)
(6, None, 643, 1)
(6, None, 643, 1)
(6, None, 643, 1)
(6, None, 648, 2)
(6,

(2, None, 537, 8)
(6, None, 543, 2)
(6, None, 544, 2)
test_phase_damping_error_canonical
(1, None, 548, 1)
(2, None, 548, 9)
(6, None, 554, 4, 1)
(6, None, 558, 2)
test_phase_damping_error_noncanonical
(1, None, 562, 1)
(2, None, 562, 10)
(6, None, 567, 2, 1)
(6, None, 568, 2, 1)
(6, None, 570, 2)
(6, None, 571, 2)
test_thermal_relaxation_error_raises_invalid_t2
(1, None, 575, 1)
(2, None, 575, 2)
(9, None, 578, 38, 0, 5)
(9, None, 580, 41, 0, 6)
test_thermal_relaxation_error_raises_invalid_t1
(1, None, 582, 1)
(2, None, 582, 2)
(9, None, 585, 38, 0, 5)
(9, None, 587, 45, 0, 6)
test_thermal_relaxation_error_raises_invalid_t1_t2
(1, None, 589, 1)
(2, None, 589, 1)
(9, None, 592, 40, 0, 5)
test_thermal_relaxation_error_t1_t2_inf_ideal
(1, None, 594, 1)
(2, None, 594, 4)
(6, None, 599, 1)
(6, None, 599, 2, 1)
test_thermal_relaxation_error_zero_time_ideal
(1, None, 602, 1)
(2, None, 602, 4)
(6, None, 607, 1)
(6, None, 607, 2, 1)
test_thermal_relaxation_error_t1_equal_t2_0state
(1, None, 61

(6, None, 46, 1)
(6, None, 46, 1)
(6, None, 47, 1)
(6, None, 48, 1)
(6, None, 49, 1)
(6, None, 50, 1)
(6, None, 51, 1)
(6, None, 52, 1)
(6, None, 53, 1)
(6, None, 56, 1)
(6, None, 56, 1)
(6, None, 57, 1)
(6, None, 57, 1)
grovers_counts
(1, None, 66, 1)
(2, None, 66, 7)
teleport_circuit
(1, None, 80, 1)
(2, None, 80, 18)
(6, None, 94, 1)
(6, None, 95, 1)
(6, None, 95, 1)
(6, None, 97, 1)
(6, None, 97, 1)
(6, None, 98, 1)
(6, None, 99, 1)
(6, None, 99, 1)
(6, None, 100, 1)
(6, None, 100, 1)
(6, None, 101, 1)
(6, None, 102, 1)
(6, None, 103, 1)
(6, None, 103, 1)
teleport_counts
(1, None, 109, 1)
(2, None, 109, 7)
..\..\subjects\tmp-repositories\qiskit-aer\test/terra/reference/ref_conditionals.py
conditional_circuits_1bit
(1, None, 20, 0)
(2, None, 20, 42)
(6, None, 43, 1)
(6, None, 43, 1)
(6, None, 64, 1)
(6, None, 64, 1)
conditional_counts_1bit
(1, None, 76, 1)
(2, None, 76, 11)
conditional_statevector_1bit
(1, None, 100, 1)
(2, None, 100, 6)
conditional_circuits_2bit
(1, None, 118, 0)
(

(6, None, 70, 1)
(13, None, 81, 2)
(6, None, 81, 1)
(13, None, 92, 2)
(6, None, 92, 1)
(13, None, 103, 2)
(6, None, 103, 1)
(13, None, 114, 2)
(6, None, 114, 1)
unitary_gate_counts_real_deterministic
(1, None, 119, 1)
(2, None, 119, 14)
unitary_gate_statevector_real_deterministic
(1, None, 150, 1)
(2, None, 150, 8)
unitary_gate_unitary_real_deterministic
(1, None, 168, 1)
(2, None, 168, 26)
(6, None, 172, 2, 1)
(6, None, 177, 2, 1)
(6, None, 182, 2, 1)
(6, None, 187, 2, 1)
(6, None, 192, 2, 1)
(6, None, 197, 2, 1)
unitary_gate_circuits_complex_deterministic
(1, None, 204, 0)
(2, None, 204, 61)
(6, None, 214, 2, 1)
(6, None, 215, 2, 1)
(13, None, 226, 2)
(6, None, 226, 1)
(13, None, 236, 2)
(6, None, 236, 1)
(13, None, 247, 2)
(6, None, 247, 1)
(13, None, 258, 2)
(6, None, 258, 1)
(13, None, 269, 2)
(6, None, 269, 1)
(13, None, 280, 2)
(6, None, 280, 1)
unitary_gate_counts_complex_deterministic
(1, None, 285, 1)
(2, None, 285, 14)
unitary_gate_statevector_complex_deterministic
(1, None,

Hash = cf5d8027d5a70f45751f1369f548b65c666442e1
..\..\subjects\tmp-repositories\qiskit-aer\src/simulators/statevector/qubitvector.hpp
..\..\subjects\tmp-repositories\qiskit-aer\src/simulators/statevector/qubitvector.hpp
An exception occurred. The file is not a python file
6e4b479700f04ccf1a421b819e88fc2218dcdbd3
https://github.com/Qiskit/qiskit-aer.git
qiskit-aer
Hash = 6e4b479700f04ccf1a421b819e88fc2218dcdbd3
..\..\subjects\tmp-repositories\qiskit-aer\src/simulators/qasm/qasm_controller.hpp
..\..\subjects\tmp-repositories\qiskit-aer\src/simulators/qasm/qasm_controller.hpp
An exception occurred. The file is not a python file
..\..\subjects\tmp-repositories\qiskit-aer\src/simulators/statevector/qubitvector.hpp
..\..\subjects\tmp-repositories\qiskit-aer\src/simulators/statevector/qubitvector.hpp
An exception occurred. The file is not a python file
..\..\subjects\tmp-repositories\qiskit-aer\src/simulators/statevector/statevector_state.hpp
..\..\subjects\tmp-repositories\qiskit-aer\src/sim

In [9]:
bug_smells.head()

,bug_id,bug_type,project_full_name,file_name,function_name,code_smell_metric,code_smell_metric_value
0,1,Classical,qiskit-ignis,circuits.py,state_tomography_circuits,PAR,2
1,1,Classical,qiskit-ignis,circuits.py,state_tomography_circuits,MLOC,1
2,1,Classical,qiskit-ignis,circuits.py,process_tomography_circuits,PAR,2
3,1,Classical,qiskit-ignis,circuits.py,process_tomography_circuits,MLOC,1
4,1,Classical,qiskit-ignis,circuits.py,_tomography_circuits,PAR,2


In [10]:
bug_smells.shape

(41378, 7)

In [11]:
bug_smells.to_csv(r'..\data\generated\code-smell-metrics-for-buggy-data.csv', index=False)

In [12]:
bug_smells.describe()

,bug_id,bug_type,project_full_name,file_name,function_name,code_smell_metric,code_smell_metric_value
count,41378,41378,41378,41378,41378,41378,41378
unique,88,2,14,414,4278,17,118
top,75,Classical,pennylane,circuit_test.py,__init__,LEC,1
freq,3203,21088,13970,1721,1200,9730,18140


In [13]:
print(bug_smells['project_full_name'].unique())

['qiskit-ignis' 'qiskit-aer' 'pennylane' 'ProjectQ' 'QuantumLibraries'
 'OpenQL' 'Cirq' 'amazon-braket-sdk-python' 'mitiq' 'tequila'
 'strawberryfields' 'qiskit-terra' 'pyquil' 'dwave-system']
